Climbers notoriously favor one style of climbing over another. Lighter climbers like delicate technical face routes. Stronger, heavier climbers excel at overhung routes with lots of big holds they can wrestle with. Nobody likes slab. Crack is a unique type that requires some degree of specialization (and pain tolerance). 

This notebook takes route tick data and scrapes the descriptions of those routes for keywords indicating it's a certain type of route. Returns arrays with types of climbs and the frequencies of them to feed into a pie chart eventually.

In [1]:
import sqlite3 as sl
import re
from datetime import datetime
import pandas as pd

#con = sl.connect('my-test2.db')

In [6]:
#redone word search function

def findCommonWords(description): # take in a string
    
    keywords = ['crack','offwidth','off-width','chimney','face','slab','crimp','overhang','overhung','roof']
    counts = [0]*len(keywords)
    
    #search the string for occurrences of the climb words 
    re.findall('crack', description)
    
    for word in keywords: #for each keyword
        counts[keywords.index(word)] = len(re.findall(word, description)) #count frequencies
    
    types = []
    crackIndex = 3 # where the crack words stop
    faceIndex = 6 # where the face words stop
    overhangIndex = len(counts) # where the overhang words stop
    
    # reduce from list of words to a single descriptor (crack/offwidth/off-width/chimney => 'Crack')
    types.append(sum(counts[0:crackIndex + 1])) # append crack word counts
    types.append(sum(counts[crackIndex + 1: faceIndex + 1])) # append face word counts
    types.append(sum(counts[faceIndex + 1: overhangIndex + 1]))
        
    return types

def getClimbType(description):
    climbType = {0 : 'Crack', 1: 'Face', 2: 'Overhang'}
    counts = findCommonWords(description)
    if max(counts) > 0:
        index = [i for i, j in enumerate(counts) if j == max(counts)][0] #find index of most used word
        return climbType[index]
    else: 
        return 'None'
    

calculating the styles with the above functions..

In [7]:
with con:
    df = pd.read_sql_query('SELECT description FROM routes;', con)
    
typeList = []
for desc in df.description:
    climbType = getClimbType(desc)
    typeList.append(climbType)

In [8]:
with con: 
    con.execute("ALTER TABLE routes ADD COLUMN style TEXT;")

adding a column for styles

with con: 
    con.execute("ALTER TABLE routes ADD COLUMN style TEXT;")
    
only need to run once.

couldn't get executemany to work for some reason. it worked before. whatever. using a for loop to insert all the climb styles into the route db

In [10]:
n = 0
with con:
    while n < len(typeList):
        if n % 20000 == 0:
            print(n, typeList[n])
        con.execute('update routes set style = ? where rowid = ?', (typeList[n], n))
        n += 1

0 Crack
20000 Crack
40000 Crack
60000 Face
80000 None
100000 Crack
120000 None
140000 None
160000 Crack
180000 Face
200000 Face
220000 Face
240000 Crack


styles are updated in the DB. what now? work on the pie charts
you want one for each grade range ? 

1. divide the df into grades
2. count the different styles
3. submit for each

In [1]:
### code from a previous notebook to load tick data

import sqlite3 as sl
from datetime import datetime
import pandas as pd

# small utility function to help sort the dates in grade dictionary
def Sort_Tuple(tup):  
    tup.sort(key = lambda x: x[0])  
    return tup 

def monthlyPBs(df):
    monthlyPBs = {} # monhigh is a list of months and monthly PB
    for index, row in df.iterrows(): # for every climb in the df
        month = row.monthyear
        grade = row.gradekey
        
        if month not in monthlyPBs.keys(): # if it's the first climb of the month
            monthlyPBs[month] = grade # set the high of the month to the climb grade
        else: # if it's not the first
            highgrade = monthlyPBs[month] # get the current PB of the month to compare
            if grade > highgrade: # if the current PB is higher
                monthlyPBs[month] = grade # set a new current PB 

        if str(row.grade) in gradeToNum.keys():
            df.at[index, 'gradekey'] = gradeToNum[str(row.grade)]
        else:
            df.at[index,'gradekey'] = 0.0
    return monthlyPBs    

con = sl.connect('my-test2.db')
with con:
    df = pd.read_sql_query('SELECT ticks.url, grade, date, status, style FROM ticks INNER JOIN routes ON routes.url = ticks.url where ticks.rowid < 100;', con)
    
df.loc[df['grade'] == 5.1, 'grade'] = '5.10b/c'
df.loc[df['grade'] == 5.11, 'grade'] = '5.11b/c'
df.loc[df['grade'] == 5.12, 'grade'] = '5.12b/c'

sendConditions = ['Lead / Onsight', 'Lead / Flash', 'Lead / Redpoint', 'Lead / Pinkpoint', 'Lead', 'Solo']

# only sends, may also filter out boulders as a side effect
df = df.loc[df['status'].isin(sendConditions)] 

gradeToNum = {
    '5.0' : 0, 'easy' : 0, 'Easy' : 0, '5.1' : 1, '5.2' : 2, '5.3' :  3, '5.4' : 4,
    '5.5' : 5, '5.6' : 6, '5.7-' : 7, '5.7' : 7, '5.7+' : 7, '5.8-' : 8, '5.8' : 8, 
    '5.8+' : 8, '5.9-' : 9, '5.9' : 9, '5.9+' : 9,'5.10a' : 10, '5.10-' : 10, 
    '5.10a/b' : 10, '5.10b' : 11, '5.10b/c' : 12, '5.10' : 12, '5.10c' : 13, 
    '5.10d' : 14, '5.10+' : 14, '5.10c/d' : 14, '5.11a' : 15, '5.11-' : 15, '5.11a/b' : 15,
    '5.11b' : 16,'5.11b/c' : 17, '5.11' : 17, '5.11c' : 18, '5.11d' : 19, '5.11+' : 19, 
    '5.11c/d' : 19, '5.12a' : 20, '5.12-' : 20, '5.12a/b' : 20, '5.12b' : 21, '5.12b/c' : 22, 
    '5.12' : 22, '5.12c' : 23, '5.12d' : 24, '5.12+' : 24, '5.12c/d' : 24, '5.13a' : 25, 
    '5.13-' : 25, '5.13a/b': 25, '5.13b': 26, '5.13b/c': 27, '5.13': 27, '5.13c': 28, 
    '5.13d': 29, '5.13+': 29, '5.13c/d': 29, '5.14a': 30, '5.14-': 30, '5.14a/b': 30, '5.14b': 31,
    '5.14b/c': 32, '5.14': 32, '5.14c': 33, '5.14d': 34, '5.14+': 34, '5.14c/d': 34,
    '5.15a': 35, '5.15-': 35, '5.15a/b': 35, '5.15b': 36, '5.15b/c': 37, '5.15': 37, '5.15c': 38,
    '5.15d': 39, '5.15+': 39, '5.15c/d': 39, '5.16a': 40, '5.16-': 40, '5.16a/b': 40, '5.16b': 41,
    '5.16b/c': 41, '5.16': 41, '5.16c': 42
}

monthyear = []
gradekey = []

for index, row in df.iterrows(): # for every tick in the ticklist
    string = datetime.strptime(row.date, '%b %d, %Y')
    month = string.strftime('%Y-%m')
    monthyear.append(month) # add monthyear value

    if str(row.grade) in gradeToNum.keys(): # also, for every tick, if the grade is in the translator
        grade = gradeToNum[str(row.grade)] # insert the new gradekey in the column
        gradekey.append(grade)
    else:
        grade = -1 # if it's not in the translator it's probably an iceclimb or something, skip it
        gradekey.append(grade)
        
df['monthyear'] = monthyear
df['gradekey'] = gradekey

monthlyPBs = monthlyPBs(df)
highlist = list(monthlyPBs.items())

highlist = Sort_Tuple(highlist)
datefill = pd.date_range(highlist[0][0],highlist[-1][0], freq='MS').strftime("%Y-%m").tolist()

In [39]:
df

,url,grade,date,status,monthyear,gradekey
1,https://www.mountainproject.com/route/10580357...,5.11a,"Mar 9, 2021",Solo,2021-03,15
3,https://www.mountainproject.com/route/11118300...,5.11c,"Nov 28, 2020",Lead / Onsight,2020-11,18
4,https://www.mountainproject.com/route/11194545...,5.10b,"Oct 19, 2020",Lead / Onsight,2020-10,11
5,https://www.mountainproject.com/route/10588924...,5.10a/b,"Oct 17, 2020",Lead / Onsight,2020-10,10
6,https://www.mountainproject.com/route/10579178...,5.10a,"Oct 17, 2020",Lead / Onsight,2020-10,10
...,...,...,...,...,...,...
86,https://www.mountainproject.com/route/10592414...,5.4,"Aug 4, 2018",Lead / Onsight,2018-08,4
89,https://www.mountainproject.com/route/10588619...,5.10c,"Jun 30, 2018",Lead / Redpoint,2018-06,13
96,https://www.mountainproject.com/route/10640302...,5.11c,"Jun 30, 2018",Lead / Flash,2018-06,18
97,https://www.mountainproject.com/route/10627455...,5.11b,"Jun 30, 2018",Lead / Redpoint,2018-06,16


In [16]:
# filtering...
with con:
    df = pd.read_sql_query('SELECT ticks.url, grade, date, status, style FROM ticks INNER JOIN routes ON routes.url = ticks.url where ticks.rowid < 100;', con)
    
sendConditions = ['Lead / Onsight', 'Lead / Flash', 'Lead / Redpoint', 'Lead / Pinkpoint', 'Lead', 'Solo']

# only sends, may also filter out boulders as a side effect
df = df.loc[df['status'].isin(sendConditions)] 

In [33]:
# sectioning climbs out into different grade boxes. to help analyze if styles change for easier or more difficult grades.

nine = df[['grade','style']][df.grade.str.match(r'(^5.9.*)')==True]
ten = df[['grade','style']][df.grade.str.match(r'(^5.10.*)')==True]
ele = df[['grade','style']][df.grade.str.match(r'(^5.11.*)')==True]
twelve = df[['grade','style']][df.grade.str.match(r'(^5.12.*)')==True]

ninedict = nine['style'].value_counts().to_dict()
tendict = ten['style'].value_counts().to_dict()
eledict = ele['style'].value_counts().to_dict()
twelvedict = twelve['style'].value_counts().to_dict()

ninecounts = ninedict.values()
ninelabels = ninedict.keys()

tencounts = tendict.values()
tenlabels = tendict.keys()

elecounts = eledict.values()
elelabels = eledict.keys()

twelvecounts = twelvedict.values()
twelvecounts = twelvedict.keys()

In [37]:
tencounts

dict_values([6, 5, 3, 3])